# JSON & HTML 형식으로 데이터 수집 & 저장

* **종합부동산세법** : JSON
* **종합부동산세 판례** : HTML

## 종합부동산세법

In [1]:
import requests
import json
import os

# 1. API 정보 입력
API_KEY = "unknownlimitless0301"
MST = "269879"  # 종합부동산세법
url = f"https://www.law.go.kr/DRF/lawService.do?OC={API_KEY}&target=law&MST={MST}&type=JSON"

# 2. 저장 경로
save_dir = r"C:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\data_json\law"
os.makedirs(save_dir, exist_ok=True)
file_path = os.path.join(save_dir, "종합부동산세법.json")

# 3. GET 요청 보내기
response = requests.get(url)

# 4. 응답 상태 확인 & 저장
if response.status_code == 200:
    data = response.json()
    
    # 5. JSON 파일로 저장
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    
    print(f"✅ 저장 완료: {file_path}")

else:
    print(f"❌ 요청 실패 (상태코드: {response.status_code})")

✅ 저장 완료: C:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\data_json\law\종합부동산세법.json


## 판례

### 판례 목록 조회하여 JSON 형식으로 저장
* 판례명에 '종합부동산세'라는 단어가 없더라도, 실제로 종합부동산세법을 근거법령으로 삼는 판례 목록 수집

* **query**
    * **종합부동산세법** 으로 설정 : 945건
    * **종합부동산세** 로 설정 : 1263건
    * **종부세** 로 설정 : 66건

In [2]:
import requests
import json
import os

# 1. 기본 정보 입력
API_KEY = "unknownlimitless0301"
query = "종부세"
base_url = "https://www.law.go.kr/DRF/lawSearch.do"

# 2. 저장 디렉토리 설정
save_dir = r"C:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\data_json\case\case_list"
os.makedirs(save_dir, exist_ok=True)
file_path = os.path.join(save_dir, "종부세_판례목록.json")

# 3. 요청 파라미터
params = {
    "OC": API_KEY,
    "target": "prec",
    "type": "JSON",
    "search": 2,              # 2: 본문 전체 검색
    "query": query,
    "display": 100,           # 최대 100개까지 한 번에
    "page": 1                 # 페이지 번호
}

all_results = []
while True:
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        print(f"❌ 요청 실패 (상태코드: {response.status_code})")
        break

    data = response.json()
    items = data.get("PrecSearch", {}).get("prec", [])

    if not items:
        break

    all_results.extend(items)
    print(f"📄 {len(items)}건 수집 (page={params['page']})")
    
    if len(items) < params["display"]:
        break  # 마지막 페이지
    else:
        params["page"] += 1  # 다음 페이지로 이동

# 4. JSON 파일로 저장
with open(file_path, "w", encoding="utf-8") as f:
    json.dump(all_results, f, ensure_ascii=False, indent=2)

print(f"\n✅ 판례 목록 저장 완료: {file_path}")

📄 66건 수집 (page=1)

✅ 판례 목록 저장 완료: C:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\data_json\case\case_list\종부세_판례목록.json


#### 중복되는 판례일련번호에 대해 중복 제거('종합부동산세', '종합부동산세법', '종부세' 각각을 수집 후 중복 제거)

In [3]:
import requests
import json
import time
import os

# 사용자 API ID와 검색어 설정
API_KEY = "unknownlimitless0301"
keywords = ["종합부동산세", "종합부동산세법", "종부세"]
base_url_list = "https://www.law.go.kr/DRF/lawSearch.do"

def fetch_case_list(keyword):
    """검색어에 대한 판례 목록을 모두 가져오는 함수"""
    params = {
        "OC": API_KEY,
        "target": "prec",
        "type": "JSON",   # JSON 형식으로 응답을 받음
        "search": 2,      # 2: 본문 검색
        "query": keyword,
        "display": 100,   # 한 번에 최대 100건
        "page": 1
    }
    all_items = []
    while True:
        resp = requests.get(base_url_list, params=params)
        resp.raise_for_status()
        data = resp.json()
        items = data.get("PrecSearch", {}).get("prec", [])
        if not items:
            break
        all_items.extend(items)
        # 마지막 페이지가 아니면 page 번호를 증가
        if len(items) < params["display"]:
            break
        params["page"] += 1
        time.sleep(0.3)
    return all_items

# 여러 키워드로 목록을 가져와 판례일련번호 기준으로 중복 제거
case_dict = {}
for kw in keywords:
    results = fetch_case_list(kw)
    for item in results:
        # 같은 판례일련번호가 나오면 덮어쓰기
        case_dict[item["판례일련번호"]] = item

# 중복 제거 후의 판례 건수 출력
unique_count = len(case_dict)
print(f"중복 제거 후 판례 건수: {unique_count}개")

# 목록을 파일로 저장
with open("종합부동산세_판례목록_통합.json", "w", encoding="utf-8") as f:
    json.dump(list(case_dict.values()), f, ensure_ascii=False, indent=2)

중복 제거 후 판례 건수: 1268개


### 조회한 판례들을 HTML 형식으로 저장

In [4]:
# case
import requests
import json
import time
import os
from typing import Dict, Optional

class CaseContentCollector:
    def __init__(self, api_key: str, base_path: str):
        self.api_key = api_key
        self.base_url = "http://www.law.go.kr/DRF/lawService.do"
        self.base_path = base_path
        self.success_count = 0
        self.fail_count = 0
        self.failed_cases = []
        os.makedirs(base_path, exist_ok=True)

    def fetch_case_html(self, case_id: str) -> Optional[str]:
        """판례일련번호로 HTML 본문을 가져오는 함수"""
        params = {
            "OC": self.api_key,
            "target": "prec",
            "type": "HTML",  # HTML 형식으로 요청
            "ID": case_id
        }
        try:
            response = requests.get(self.base_url, params=params, timeout=30)
            response.raise_for_status()
            if response.text and len(response.text) > 100:
                return response.text
            else:
                print(f"빈 응답 또는 너무 짧은 응답: {case_id}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"네트워크 오류 - 판례 {case_id}: {e}")
            return None

    def save_case_html(self, case_info: Dict, html_content: str) -> bool:
        """판례 HTML을 파일로 저장"""
        try:
            case_id = case_info.get("판례일련번호", "")
            case_number = case_info.get("사건번호", "").replace("/", "-").replace("\\", "-")
            # 안전한 파일명 생성: 공백, 점, 하이픈, 밑줄만 허용
            safe_case_number = "".join(c for c in case_number if c.isalnum() or c in " .-_")
            filename = f"{case_id}_{safe_case_number}.html" if safe_case_number else f"{case_id}.html"
            filepath = os.path.join(self.base_path, filename)
            with open(filepath, "w", encoding="utf-8") as f:
                f.write(html_content)
            return True
        except Exception as e:
            print(f"파일 저장 오류 - 판례 {case_info.get('판례일련번호', 'unknown')}: {e}")
            return False

    def collect_all_cases(self, case_list_file: str, delay: float = 0.5) -> None:
        """모든 판례 본문을 HTML로 수집하는 메인 함수"""
        try:
            with open(case_list_file, "r", encoding="utf-8") as f:
                case_list = json.load(f)
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {case_list_file}")
            return
        except json.JSONDecodeError:
            print(f"❌ JSON 파일 형식이 올바르지 않습니다: {case_list_file}")
            return

        total_cases = len(case_list)
        print(f"📋 총 {total_cases}건의 판례 본문을 HTML로 수집을 시작합니다.")
        print(f"💾 저장 경로: {self.base_path}")
        print(f"⏱️ 요청 간격: {delay}초\n")

        for i, case_info in enumerate(case_list):
            case_id = case_info.get("판례일련번호", "")
            case_number = case_info.get("사건번호", "")
            if not case_id:
                print(f"[{i+1}/{total_cases}] ⚠️ 판례일련번호가 없습니다")
                self.fail_count += 1
                continue
            print(f"[{i+1}/{total_cases}] 🔍 판례 {case_id} ({case_number}) 수집 중...")

            html_content = self.fetch_case_html(case_id)
            if html_content:
                if self.save_case_html(case_info, html_content):
                    self.success_count += 1
                    print(f" ✅ 수집 및 저장 성공")
                else:
                    self.failed_cases.append(case_id)
                    self.fail_count += 1
                    print(f" ❌ 저장 실패")
            else:
                self.failed_cases.append(case_id)
                self.fail_count += 1
                print(f" ❌ 수집 실패")

            time.sleep(delay)

            # 100건마다 진행상황 출력
            if (i + 1) % 100 == 0:
                processed = i + 1
                success_rate = (self.success_count / processed) * 100
                print(f"\n📊 --- 진행상황 ({processed}/{total_cases}) ---")
                print(f"✅ 성공: {self.success_count}건, ❌ 실패: {self.fail_count}건")
                print(f"📈 성공률: {success_rate:.1f}%\n")

        self._print_summary(total_cases)
        self._save_failed_list()

    def _print_summary(self, total: int) -> None:
        """수집 결과 요약 출력"""
        print(f"\n{'='*70}")
        print(f"🎉 판례 HTML 본문 수집 완료")
        print(f"{'='*70}")
        print(f"📋 전체 대상: {total:,}건")
        print(f"✅ 수집 성공: {self.success_count:,}건 ({(self.success_count/total)*100:.1f}%)")
        print(f"❌ 수집 실패: {self.fail_count:,}건 ({(self.fail_count/total)*100:.1f}%)")
        print(f"💾 저장 경로: {self.base_path}")
        if self.failed_cases:
            print(f"\n⚠️ 실패한 판례번호 (처음 10개):")
            for case_id in self.failed_cases[:10]:
                print(f" - {case_id}")
            if len(self.failed_cases) > 10:
                print(f" ... 외 {len(self.failed_cases)-10}건")

    def _save_failed_list(self) -> None:
        """실패한 판례 목록을 파일로 저장"""
        if self.failed_cases:
            failed_file = os.path.join(self.base_path, "failed_cases.json")
            with open(failed_file, "w", encoding="utf-8") as f:
                json.dump(self.failed_cases, f, ensure_ascii=False, indent=2)
            print(f"\n📄 실패 목록 저장: {failed_file}")

def main():
    """메인 실행 함수"""
    API_KEY = "unknownlimitless0301"
    BASE_PATH = r"C:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트"
    CASE_LIST_FILE = os.path.join(BASE_PATH, "종합부동산세_판례목록_통합.json")
    HTML_SAVE_PATH = os.path.join(BASE_PATH, "data_json_html", "case")

    print("🚀 === 종합부동산세 판례 HTML 본문 수집 시작 ===")
    print(f"🔑 API 키: {API_KEY}")
    print(f"📂 목록 파일: {CASE_LIST_FILE}")
    print(f"💾 저장 경로: {HTML_SAVE_PATH}\n")

    collector = CaseContentCollector(API_KEY, HTML_SAVE_PATH)
    collector.collect_all_cases(CASE_LIST_FILE, delay=0.5)

if __name__ == "__main__":
    main()


🚀 === 종합부동산세 판례 HTML 본문 수집 시작 ===
🔑 API 키: unknownlimitless0301
📂 목록 파일: C:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\종합부동산세_판례목록_통합.json
💾 저장 경로: C:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\data_json_html\case

📋 총 1268건의 판례 본문을 HTML로 수집을 시작합니다.
💾 저장 경로: C:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\data_json_html\case
⏱️ 요청 간격: 0.5초

[1/1268] 🔍 판례 606317 (전주지방법원-2021-구합-1678) 수집 중...
 ✅ 수집 및 저장 성공
[2/1268] 🔍 판례 604957 (대법원-2024-두-66518) 수집 중...
 ✅ 수집 및 저장 성공
[3/1268] 🔍 판례 605331 (2024스866) 수집 중...
 ✅ 수집 및 저장 성공
[4/1268] 🔍 판례 499163 (대법원-2024-두-59381) 수집 중...
 ✅ 수집 및 저장 성공
[5/1268] 🔍 판례 605495 (서울행정법원-2023-구합-72950) 수집 중...
 ✅ 수집 및 저장 성공
[6/1268] 🔍 판례 603625 (서울행정법원-2024-구합-66419) 수집 중...
 ✅ 수집 및 저장 성공
[7/1268] 🔍 판례 342351 (대구고등법원-2024-누-12215) 수집 중...
 ✅ 수집 및 저장 성공
[8/1268] 🔍 판례 606133 (수원지방법원-2022-구합-76451) 수집 중...
 ✅ 수집 및 저장 성공
[9/1268] 🔍 판례 499239 (대법원-2024-두-55938) 수집 중...
 ✅ 수집 및 저장 성공
[10/1268] 🔍 판례 242301 (서울고등법원-2

#### 개별 판례 파일들을 하나의 HTML 파일로 통합합하여 저장

In [2]:
import os
import re

html_dir = r"C:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\data_json_html\case"
output_file = os.path.join(html_dir, "merged_cases.html")

def extract_prec_seq(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
        match = re.search(r'id="precSeq"\s+value\s*=\s*"(\d+)"', content)
        if match:
            return match.group(1)
    return None

html_head = '''<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <title>통합 판례 모음</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; }
        .case-block { margin-bottom: 40px; }
        iframe { width: 100%; height: 600px; border: 1px solid #ccc; }
    </style>
</head>
<body>
    <h1>판례 통합 문서 (총 1268건)</h1>
'''

html_tail = '''
</body>
</html>
'''

iframe_blocks = ""

for filename in sorted(os.listdir(html_dir)):
    if filename.endswith(".html"):
        filepath = os.path.join(html_dir, filename)
        prec_seq = extract_prec_seq(filepath)
        if prec_seq:
            title = filename.replace(".html", "")
            iframe = f'''
    <div class="case-block">
        <h2>{title}</h2>
        <iframe src="https://www.law.go.kr/LSW/precInfoP.do?precSeq={prec_seq}&mode=0" loading="lazy"></iframe>
    </div>
'''
            iframe_blocks += iframe

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(html_head + iframe_blocks + html_tail)

print(f"✅ 통합 완료: {output_file}")

✅ 통합 완료: C:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\data_json_html\case\merged_cases.html
